In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
!pip install yfinance
import yfinance as yf
!pip install yahoofinancials
import yahoofinancials

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=8ffa7542e1359d76bf582dbdf9993113715d69c7c1627d16e4e12105fcb13050
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance
  Created wheel for yahoofinancials: filename=yahoofinancials-1.5-cp36-none-any.whl size=15195 sha256=3d834e087da5a5819289f3f73a95426815a727405fb62bc173e436b0d364d068
  Stored in directory: /root/.cache/pip/wheels/42/15/e7/240c5cdc8782b9479ba73404640a3be57bb4053a5c3edd6be5
Successfully built yahoofinancials


In [0]:
stock_name = ["NYSE COMPOSITE (DJ)","NASDAQ Composite","Japan Exchange Group, Inc.","London Stock Exchange Group plc","Shanghai Stock Exchange","Hong Kong Exchanges","S&P BSE SENSEX","NIFTY 50"]

In [0]:
stock_code = ["^NYA","^IXIC","JPXGY","LSE.L","^SSEC","0388.HK","^BSESN","^NSEI" ]

In [0]:
currency = ["USD","USD","USD","GBp","CNY","HKD","INR","INR"]

In [0]:
data = pd.DataFrame(columns=["stock_name","stock_code","currency"]) 

In [0]:
data.stock_code = stock_code
data.stock_name = stock_name
data.currency = currency

In [67]:
data

,stock_name,stock_code,currency
0,NYSE COMPOSITE (DJ),^NYA,USD
1,NASDAQ Composite,^IXIC,USD
2,"Japan Exchange Group, Inc.",JPXGY,USD
3,London Stock Exchange Group plc,LSE.L,GBp
4,Shanghai Stock Exchange,^SSEC,CNY
5,Hong Kong Exchanges,0388.HK,HKD
6,S&P BSE SENSEX,^BSESN,INR
7,NIFTY 50,^NSEI,INR


In [76]:
k = 0
tableau_data = pd.DataFrame(columns=["Date","Close","Name"])
for i in range(data['stock_code'].shape[0]):
  temp_data = yf.download(data.loc[i,"stock_code"], start='2019-11-01', end='2020-04-10', progress=False)
  if data.loc[i,"currency"] == "GBp":
    temp_data["Close"] = 1.25*temp_data["Close"]
  elif data.loc[i,"currency"] == "CNY":
    temp_data["Close"] = 0.14*temp_data["Close"]
  elif data.loc[i,"currency"] == "HKD":
    temp_data["Close"] = 0.13 *temp_data["Close"]
  elif data.loc[i,"currency"] == "INR":
    temp_data["Close"] = 0.013 *temp_data["Close"]
    
  if temp_data.any != None:
    temp_data['Date'] = temp_data.index
    temp_data.reset_index(drop=True,inplace= True)
    temp_data = temp_data[["Date","Close"]]
    temp_data["Name"] = data.loc[i,"stock_name"]
    
    for index, row in temp_data.iterrows():
  
      tableau_data.loc[index+k,"Date"] = row['Date']
      tableau_data.loc[index+k,"Close"] = row['Close']
      tableau_data.loc[index+k,"Name"] = row['Name']
      k=k+index
  else:
    print("not found")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1 Failed download:
- ^SSEC: No data found for this date range, symbol may be delisted


In [0]:
tableau_data.to_csv("tableau_data.csv")

In [34]:
temp_data

,Date,Close,Name
Date,,,
2019-11-01,2019-11-01,11890.599609,^NSEI
2019-11-04,2019-11-04,11941.299805,^NSEI
2019-11-05,2019-11-05,11917.200195,^NSEI
2019-11-06,2019-11-06,11966.049805,^NSEI
2019-11-07,2019-11-07,12012.049805,^NSEI
...,...,...,...
2020-04-01,2020-04-01,8253.799805,^NSEI
2020-04-03,2020-04-03,8083.799805,^NSEI
2020-04-07,2020-04-07,8792.200195,^NSEI


In [0]:
# scraping data from world-o-meter for coronavirus cases

In [0]:
res = requests.get("https://www.worldometers.info/coronavirus/")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[1] 
df = pd.read_html(str(table))[0]


In [0]:
df.to_csv("current_cases.csv")